# 01: Basic Hierarchical Bayesian Calibration

**Difficulty:** Beginner | **Time:** ~10 minutes

## Overview

This notebook demonstrates a complete end-to-end hierarchical Bayesian calibration workflow for the biofilm model.

### What You'll Learn

- How to set up and run a basic calibration
- Understanding the M1 → M2 → M3 hierarchical approach
- Visualizing and interpreting posterior distributions
- Comparing estimated parameters with ground truth

### Workflow

1. Import libraries and configure settings
2. Generate synthetic observation data
3. Run hierarchical calibration (M1 → M2 → M3)
4. Analyze results and visualize posteriors
5. Validate parameter estimates

## 1. Setup

In [ ]:
# Add parent directory to path to import src modules
import sys
sys.path.insert(0, '..')

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Project imports
from src.config import CONFIG, DEBUG, get_theta_true
from src.hierarchical import hierarchical_case2
from src.viz_paper import plot_corner, plot_timeseries

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
output_dir = Path('figures')
output_dir.mkdir(exist_ok=True)

print("✓ Setup complete")
print(f"  DEBUG mode: {DEBUG}")
print(f"  N0 (samples): {CONFIG['N0']}")
print(f"  Output dir: {output_dir}")

## 2. Problem Setup

### The Biofilm Model

We're calibrating a 14-dimensional parameter vector for a multi-species biofilm model:

- **M1 (Coarse)**: θ[0:5] - Species 1-2 interactions
- **M2 (Medium)**: θ[5:10] - Species 3-4 interactions  
- **M3 (Fine)**: θ[10:14] - Cross-species interactions

### True Parameters (Ground Truth)

These are the "true" parameters we're trying to recover:

In [ ]:
# Get true parameter vector
theta_true = get_theta_true()

print("True Parameters:")
print("="*50)
print(f"M1 (θ[0:5]):   {theta_true[0:5]}")
print(f"M2 (θ[5:10]):  {theta_true[5:10]}")
print(f"M3 (θ[10:14]): {theta_true[10:14]}")
print("="*50)
print(f"Total: {len(theta_true)} parameters")

## 3. Run Hierarchical Calibration

The calibration proceeds in three stages:

1. **M1**: Calibrate θ[0:5] using species 1-2 data
2. **M2**: Calibrate θ[5:10] using species 3-4 data (with M1 posterior as prior)
3. **M3**: Calibrate θ[10:14] using all species data (with M1+M2 posteriors as prior)

Each stage uses TMCMC (Transitional Markov Chain Monte Carlo) for sampling.

In [ ]:
print("Starting hierarchical Bayesian calibration...")
print("This may take a few minutes in DEBUG mode, longer in production mode.\n")

start_time = time.time()

# Run hierarchical calibration
results = hierarchical_case2(CONFIG)

elapsed_time = time.time() - start_time

print(f"\n{'='*60}")
print(f"✓ Calibration complete in {elapsed_time:.1f} seconds")
print(f"{'='*60}")

## 4. Extract Results

The results object contains:
- Parameter estimates at each stage
- Posterior samples from TMCMC
- Convergence diagnostics
- RMSE and other metrics

In [ ]:
# Extract parameter estimates
theta_M1 = results.theta_M1
theta_M2 = results.theta_M2  
theta_final = results.theta_final

# Extract posterior samples
samples_M1 = results.tmcmc_M1.samples[-1]
samples_M2 = results.tmcmc_M2.samples[-1]
samples_M3 = results.tmcmc_M3.samples[-1]

# Convergence flags
converged_M1 = results.converged_M1
converged_M2 = results.converged_M2
converged_M3 = results.converged_M3

print("Results Summary:")
print("="*60)
print(f"M1 samples: {samples_M1.shape}")
print(f"M2 samples: {samples_M2.shape}")
print(f"M3 samples: {samples_M3.shape}")
print(f"\nConvergence: M1={converged_M1}, M2={converged_M2}, M3={converged_M3}")
print(f"RMSE: {results.rmse:.6f}")

## 5. Compare Estimates with Ground Truth

Let's see how well we recovered the true parameters:

In [ ]:
# Compute errors
error_M1 = np.abs(theta_M1 - theta_true[0:5])
error_M2 = np.abs(theta_M2 - theta_true[5:10])
error_M3 = np.abs(theta_final[10:14] - theta_true[10:14])

# Compute standard deviations
std_M1 = np.std(samples_M1, axis=0)
std_M2 = np.std(samples_M2, axis=0)
std_M3 = np.std(samples_M3, axis=0)

print("\nM1 Parameters (Species 1-2):")
print("-" * 70)
print(f"{'Param':<10} {'True':<12} {'Estimated':<12} {'Std Dev':<12} {'Error':<10}")
print("-" * 70)
for i in range(5):
    print(f"θ[{i}]      {theta_true[i]:>10.4f}  {theta_M1[i]:>10.4f}  {std_M1[i]:>10.4f}  {error_M1[i]:>8.4f}")

print("\nM2 Parameters (Species 3-4):")
print("-" * 70)
print(f"{'Param':<10} {'True':<12} {'Estimated':<12} {'Std Dev':<12} {'Error':<10}")
print("-" * 70)
for i in range(5):
    print(f"θ[{i+5}]    {theta_true[i+5]:>10.4f}  {theta_M2[i]:>10.4f}  {std_M2[i]:>10.4f}  {error_M2[i]:>8.4f}")

print("\nM3 Parameters (Cross-interactions):")
print("-" * 70)
print(f"{'Param':<10} {'True':<12} {'Estimated':<12} {'Std Dev':<12} {'Error':<10}")
print("-" * 70)
for i in range(4):
    print(f"θ[{i+10}]   {theta_true[i+10]:>10.4f}  {theta_final[i+10]:>10.4f}  {std_M3[i]:>10.4f}  {error_M3[i]:>8.4f}")

# Overall statistics
mean_error = np.mean(np.concatenate([error_M1, error_M2, error_M3]))
max_error = np.max(np.concatenate([error_M1, error_M2, error_M3]))

print("\n" + "="*70)
print(f"Mean absolute error: {mean_error:.4f}")
print(f"Max absolute error:  {max_error:.4f}")
print("="*70)

## 6. Visualize Posterior Distributions

Let's plot the posterior distributions for M1 parameters:

In [ ]:
# Plot M1 posterior distributions
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
param_names = ['a₁₁', 'a₁₂', 'a₂₂', 'b₁', 'b₂']

for i, (ax, name) in enumerate(zip(axes, param_names)):
    # Histogram of posterior samples
    ax.hist(samples_M1[:, i], bins=30, density=True, alpha=0.7, edgecolor='black')
    
    # True value (vertical line)
    ax.axvline(theta_true[i], color='red', linestyle='--', linewidth=2, label='True')
    
    # Estimated value (mean)
    ax.axvline(theta_M1[i], color='blue', linestyle='-', linewidth=2, label='Estimate')
    
    ax.set_xlabel(name, fontsize=12)
    ax.set_ylabel('Density' if i == 0 else '')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / '01_M1_posteriors.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved to {output_dir / '01_M1_posteriors.png'}")

## 7. TMCMC Convergence Diagnostics

Check the TMCMC β schedule and Effective Sample Size (ESS):

In [ ]:
# Extract TMCMC diagnostics
beta_values = results.tmcmc_M1.beta_values
ess_values = results.tmcmc_M1.ess_values

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# β schedule
ax1.plot(beta_values, marker='o', linewidth=2)
ax1.set_xlabel('Stage', fontsize=12)
ax1.set_ylabel('β (Tempering Parameter)', fontsize=12)
ax1.set_title('TMCMC Tempering Schedule', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-0.05, 1.05)

# ESS evolution
ax2.plot(ess_values, marker='o', linewidth=2, label='ESS')
ax2.axhline(y=0.5*CONFIG['N0'], color='r', linestyle='--', label='50% Threshold')
ax2.set_xlabel('Stage', fontsize=12)
ax2.set_ylabel('Effective Sample Size', fontsize=12)
ax2.set_title('ESS Evolution', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / '01_tmcmc_diagnostics.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved to {output_dir / '01_tmcmc_diagnostics.png'}")
print(f"\nFinal β: {beta_values[-1]:.4f} (should be 1.0)")
print(f"Final ESS: {ess_values[-1]:.0f} / {CONFIG['N0']} ({ess_values[-1]/CONFIG['N0']*100:.1f}%)")

## 8. Summary and Interpretation

### What We Learned

1. **Hierarchical Approach**: Breaking the 14D problem into M1→M2→M3 stages makes calibration tractable
2. **TMCMC**: The tempering schedule (β: 0→1) helps sample complex posteriors
3. **Convergence**: Check ESS and β progression to ensure convergence
4. **Uncertainty**: Posterior distributions show parameter uncertainty

### Key Metrics

- **Parameter errors**: Compare estimates with ground truth
- **Posterior widths**: Narrow posteriors = well-constrained parameters
- **ESS**: Should stay above 50% of N0
- **RMSE**: Overall fit quality

### Next Steps

- Try `02_sensitivity_analysis.ipynb` to understand parameter influence
- Explore `05_advanced_visualization.ipynb` for publication-quality plots
- Experiment with different CONFIG settings (N0, stages, etc.)

## Exercises (Optional)

1. **Effect of sample size**: Re-run with different `CONFIG['N0']` values (200, 500, 1000). How does this affect accuracy?

2. **Production mode**: Set `DEBUG = False` and compare results. How much better are the estimates?

3. **Parameter modification**: Change `theta_true` values in `src/config.py` and see if the calibration still works.

4. **Noise sensitivity**: Modify `sigma_obs` in CONFIG and see how observation noise affects results.